In [1]:
import numpy as np
import pandas as pd

# setting
pd.options.display.max_rows = 10

#  6.1 Reading and Writing Data in Text Format

In [2]:
# read the csv using read_csv()

df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
# we could also have used read_table() and specified the delimeter

pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# read a file that contains no header row
# allow pandas to assign default column names

pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# specify name yourself
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
# suppose you want the 'message' column to be the index of the returned DataFrame
# you can use the 'index_col' attribute

names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message') # or index_col=4

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [7]:
# if you want to form a hierarchical index from multiple columns, pass a list of column number or name to 'index_col'

parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [8]:
# sometimes, a table might not have a fixed delimeter
# you can pass a REGULAR EXPRESSION as a delimeter for read_table

result = pd.read_table('examples/ex3.txt', sep='\s+') # '\s+' means any amount of whitespace

# because there was one fewer column name than the number of rows, read_table() infer the 1st column should be the index
result 

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [9]:
# you can pass in a list of row to 'skiprows' argument to skip lines that are not data

pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3]) # skip the 1st, 3rd and 4th rows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
# read a file with missing values
# by default, pandas uses a set of commonly occurring sentinels, such as NA and NULL

result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [11]:
# check if there are any null values with the isnull() method

pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [12]:
# the 'na_values' argument can take either a list or set of strings to consider missing values

result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [13]:
# different NA sentinels can be specified for each column in a dict
sentinels = {
    'message' : ['foo', 'NA'],
    'something' : ['two']
            }

pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 6.1.1 Reading Text Files in Pieces

In [14]:
# when processing very large files or figuring out the right set of arguments to correctly process a large file
# you may only want to read a small piece of a file

# before we look at a large file, we make the pandas display settings more compact
pd.options.display.max_rows = 10

In [15]:
# create data
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [16]:
# if you want to only read a small number of rows, specify that with the 'nrows' argument

pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [17]:
# to read a file in pieces, specify a 'chunksize' as a number of rows

chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker # returns a TextParser object

In [18]:
# the TextParser object allows you to iterate over the parts of the file arrcording to the chunksize

tot = pd.Series([], dtype=np.int32) # create a container

# iterate over the TextParser using a for loop
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 6.1.2 Writing Data to Text Format

In [19]:
# data can also be exported to a delimited format

data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
# using DataFrame's to_csv() method, we can write the data out to a comma-separated file

data.to_csv('examples/out.csv')

In [21]:
# other delimeters can be used
# writing to sys.stdout so it prints the text result to the console

import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [22]:
# missing values appear as empty strings in the output 
# You might want to denote them by some other sentinel value using the 'na_rep' option

data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [23]:
# with no other options specified, both the row and column labels are written
# both of these can be disable by using 'index' and 'header' arguments

data.to_csv(sys.stdout, index=False, header=False, na_rep='NULL')

one,1,2,3.0,4,NULL
two,5,6,NULL,8,world
three,9,10,11.0,12,foo


In [24]:
# you can also write only a subset of the columns, and in the order of your choosing
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'], na_rep='NA')

a,b,c
1,2,3.0
5,6,NA
9,10,11.0


In [25]:
# Series also has a to_csv() method

# create Index Object
dates = pd.date_range('1/1/2000', periods=7) # from '1/1/2000' to '7/1/2000'

# create data
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [26]:
ts.to_csv('examples/tseries.csv', header=False)

## 6.1.3 Working with Delimited Formats

In [27]:
# sometimes you might want to process data manually 
# for any file with a single-character delimiter, you can use Pythons's built-in csv module

import csv

f = open('examples/ex7.csv')
reader = csv.reader(f) # pass a opened file object to csv.reader() to create an iterator

In [28]:
# iterating through the reader() like a file yields tuples of values with any quote character removed

for line in reader:
    print(line)
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [29]:
# from here, it's up to you to do the wrangling necessary to put the data in the form that you need

# Here is an example. Let's take this step-by-step
# First, read the file into a list of lines
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

# Second, we split the lines into the header line and the data lines
header, values = lines[0], lines[1:]

# Finally, we can create a dictionary of data columns 
# using a dictionary comprehension and the expression zip(*values), which transposes rows to columns
data_dict = {h : v  for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [30]:
# CSV files come in many different flavors
# to define a new format, we define a subclass of csv.Dialect
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

# create data to test subclass 
data = {
    'a' : [1, 2, 3],
    'b' : [4, 5, 6],
    'c' : ["a", "b", "c"]
}
df = pd.DataFrame(data)
df.to_csv('examples/my_ex.csv', sep=';', header=False, index=False)

In [31]:
f = open('examples/my_ex.csv')

reader = csv.reader(f, dialect=my_dialect)
# we can also give individual csv dialect parameters as arguments to csv.reader() without having to define a subclass
# reader = csv.reader(f, delimiter=';', lineterminator='\n') # also works fine

for line in reader:
    print(line)
f.close()

['1', '4', 'a']
['2', '5', 'b']
['3', '6', 'c']


In [32]:
# to write delimited files manually, you can use csv.writer()
with open('examples/mydata.csv', 'w') as f:
    writer =  csv.writer(f, dialect=my_dialect) # accept an open, writable file object
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

## 6.1.4 JSON Data

In [33]:
# JSON (JavaScript Object Notation) has become one of the standards formats 
# for sending data by HTTP request between web browsers and other applications.
# it is a much more free-form data format than a tabular text form like CSV

# Example of a json string
obj = """
{
"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{
                "name": "Scott", 
                "age": 30, 
                "pets": ["Zeus", "Zuko"]
             },
             {
                "name": "Katie", 
                "age": 38,
                "pets": ["Sixes", "Stache", "Cisco"]
             }]
}
"""

In [34]:
# There are several Python libraries for reading and writing JSON data.
# Here we will use the built-in json module
import json

# to convert a JSON string to Python form, use json.loads()
result = json.loads(obj)
print("result's type: {}".format(type(result)))

result's type: <class 'dict'>


In [35]:
# to convert a Python object back to JSON string, use json.dumps()
asjson = json.dumps(result)

print("asjson's type: {}".format(type(asjson)))

asjson's type: <class 'str'>


In [36]:
# convert a JSON object to a DataFrame (example)

siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [37]:
# the pd.read_json() can automatically convert JSON datasets in specific arrangement into a Series or DataFrame

# the default options for pd.read_json() assume that each object in the JSON array is a row in the table
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [38]:
# if you need to export data from pandas to JSON, one way is to use the to_json() method on Series and DataFrame

print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## 6.1.5 XML and HTML: Web Scraping

In [39]:
# python has many libraries for reading and writing data in the ubiquitous HTML and XML formats
# Example include: lxml, Beautiful Soup, and html5lib,...

# pandas has a built-in function, read_html(), which uses libraries like lxml and Beautiful Soup 
# to automatically parse tables out of HTML files as DataFrame objects

# so first we have to install some additional libraries used by pd.read_html()

# please install lxml, beautifulsoup4, html5lib to your local environment before proceed

In [40]:
# the pandas.read_html() function has a number of options, 
# but by default it searches for and attempts to parse all tabular data contained within <table> tags.

tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [41]:
# the result is a list of DataFrame objects

print('tables type = ', type(tables))
print('tables length = ', len(tables))
print('tables[0] type = ', type(tables[0]))

tables type =  <class 'list'>
tables length =  1
tables[0] type =  <class 'pandas.core.frame.DataFrame'>


In [42]:
# display data
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [43]:
# as you will learn in later chapters, from here we could proceed to do some data cleaning and analysis

# Example: computing the number of bank failures by year
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

### 6.1.5.1 Parsing XML with lxml.objectify

In [44]:
# XML(eXtensible Markup Language) is another common structured data format supporting hierarchical, nested data with metadata.
# using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot()
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
with open(path) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()

In [45]:
# root.INDICATOR returns a generator yielding each <INDICATOR> XML element
# for each record, we can populate a dict of tag name to data values (excluding a few tags)

data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR: 
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [46]:
# finally, convert this list of dicts into a DataFrame
perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


# 6.2 Binary Data Formats

In [47]:
# one of the easiest way to store data (serialization) efficiently in binary is using Python's built-in "pickle" serialization

frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [48]:
# pandas objects all have a to_pickle() method that writes the data to disk in pickle format
frame.to_pickle('examples/frame_pickle')

In [49]:
# you can read any "pickled" object stored in a file by using the built-in 'pickle' directly
# or using pandas.read_pickle()
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 6.2.1 Using HDF5 Format

In [50]:
# HDF5 (hierachichal data format) is a well-regarded file format intended for storing large quantities of scientific array data
# The HDFStore class works like a dict and handle the low-level details

frame = pd.DataFrame({'a' : np.random.randn(100)})

store = pd.HDFStore('examples/data.h5')

store['obj1'] = frame
store['obj1_col'] = frame['a']

store # show the HDFStore class type && file path

<class 'pandas.io.pytables.HDFStore'>
File path: examples/data.h5

In [51]:
# Objects contained in the HDF5 file can then be retrieved with the same dict-like API
store['obj1']

,a
0,-0.205968
1,-0.918593
2,-1.304142
3,-1.148827
4,0.044389
...,...
95,0.115439
96,-0.864623
97,-0.218814
98,0.397150


In [52]:
# HDFStore supports 2 storage schemas: 'fixed' and 'table'
# 'table' is generally slower, but it supports query operations using a special syntax

store.put('obj2', frame, format='table') # equal to store['obj2'] = frame but let us select options like format
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-1.186320
11,-0.845410
12,-0.993930
13,1.620066
14,0.286926
15,-1.281994


In [53]:
store.close() # close file when done working

In [54]:
# the pandas.read_hdf() function gives you a shortcut to these tools
frame.to_hdf('examples/data.h5', 'obj3', format='table')
pd.read_hdf('examples/data.h5', 'obj3', where=['index < 5'])

,a
0,-0.205968
1,-0.918593
2,-1.304142
3,-1.148827
4,0.044389


## 6.2.2 Reading Microsoft Excel Files

In [56]:
# pandas also supports reading tabular data in Excel using either the ExcelFile class or pandas.read_excel() function

# to use ExcelFile, create an instance by passing a path to a xls or xlsx file
xlsx = pd.ExcelFile('examples/ex1.xlsx')

# data stored in a sheet can then be read into DataFrame with parse
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [ ]:
# if you are reading multiple sheets in a file, then it is faster to create an ExcelFile
# but you can also simply pass the filename to pandas.read_excel()

frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [ ]:
# to write pandas data to Excel format, you must first create an ExcelWriter class
# then write data to it using object's to_excel() method

writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1') # write data from frame to Sheet1

writer.save() # save after done working

In [ ]:
# you can also pass a file to to_excel() and avoid the ExcelWriter

frame.to_excel('examples/ex2.xlsx', 'Sheet2') # no save() needed

# 6.3 Interacting with Web APIs

In [ ]:
# many website have public APIs providing data feeds via JSON or some other format
# There are a number of ways to access these APIs from python; here we use the 'requests' package
import requests

# find the last 30 Github issues for pandas on Github
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

resp = requests.get(url) # returns a Response object

resp # 200 = OK

<Response [200]>

In [ ]:
# the Response.json() method will return a dictionary containing JSON parsed into native python object
data = resp.json() 
data[0]['title']

'BUG: pandas.errors.EmptyDataError: No columns to parse from file'

In [ ]:
# each element in data is a dictionary containing all of the data found on a Github issue page
# we can pass data directly to DataFrame and extract feilds of interest

issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,46048,BUG: pandas.errors.EmptyDataError: No columns ...,"[{'id': 34444536, 'node_id': 'MDU6TGFiZWwzNDQ0...",open
1,46047,BUG: Multiindex.equals not commutative for ea ...,"[{'id': 71268330, 'node_id': 'MDU6TGFiZWw3MTI2...",open
2,46046,ENH: Add `lsuffix=`/`rsuffix=` kwargs to `pd.m...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
3,46045,FIX: clarify warning in read_sql_query(),[],open
4,46044,BUG: wide_to_long fails/works depending on whe...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,46015,PERF: join empty frame,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
26,46014,CI: Don't run xdist if pytest.mark.single_cpu,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
27,46011,ENH: add `DataFrame.to_frame` to simplify gene...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
28,46010,DEPR: groupby nuisance warnings,"[{'id': 87485152, 'node_id': 'MDU6TGFiZWw4NzQ4...",open


# 6.4 Interacting with Databases

In [58]:
# loading data from SQL into a DataFrame is fairly straightfoward, and pandas has some functions to simplify the process

# As an example, create a SQLite database using Python's built-in sqlite3 module
import sqlite3

try: # because you can't create a table if it already exist
    query = '''
        CREATE TABLE test
        (
            a VARCHAR(20), 
            b VARCHAR(20),
            c REAL,
            d INTEGER
        )
    '''
    con = sqlite3.connect('examples/mydata.sqlite')
    con.execute(query)
    con.commit()
except:
    con = sqlite3.connect('examples/mydata.sqlite')

In [62]:
# then, insert a few rows of data
data = [
        ('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)
       ]

stmt = "INSERT INTO test VALUES (?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [63]:
# most Python SQL drivers return a list of tuples when selecting data from a table

cursor = con.execute('select * from test')
rows = cursor.fetchall()

rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [69]:
# get the table's columns name using description

cursor.description

('a', 'b', 'c', 'd')

In [ ]:
# create a DataFrame
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5


In [ ]:
con.close() # close connection after done working

In [73]:
# the previous steps are quite tedious to do every time you query the database
# the SQLAlchemy project is a popular Python SQL toolkit
# that abstracts away many of the common differences between SQL databases

# pandas has a read_sql() function that enables you to read data easily from a general SQLAlchemy connection
import sqlalchemy as sqla

# here we will connect to the same SQLite database with SQLAlchemy and read data from the table created before
db = sqla.create_engine(r'sqlite:///examples/mydata.sqlite')

pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
...,...,...,...,...
10,Tallahassee,Florida,2.60,3
11,Sacramento,California,1.70,5
12,Atlanta,Georgia,1.25,6
13,Tallahassee,Florida,2.60,3
